In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import gym

from sklearn.neighbors import KNeighborsClassifier


# Environment

#### Noisy Cart Pole

This is the noisy version of the CartPole-v0 environment of OpenAI.  
https://gym.openai.com/envs/CartPole-v0  
https://github.com/openai/gym/wiki/CartPole-v0  


In [2]:
class NoisyCartPoleEnvironment:
    
    stateDimension = 4
    actionDimension = 1
    actionSpace = range(2)
    transitionSigmas = [ 1e-2, 1e-5, 1e-2, 1e-5 ]
    
    def __init__(self):
        pass
    
    def cartpole_reset(self):
        state = np.random.uniform(low=-0.05, high=0.05, size=(4,))
        return np.array(state)
    
    # Extracted from OpenAI environment CartPole-v0
    def cartpole_step(self, state, action):

        gravity = 9.8
        masscart = 1.0
        masspole = 0.1
        total_mass = (masspole + masscart)
        length = 0.5 # actually half the pole's length
        polemass_length = (masspole * length)
        force_mag = 10.0
        tau = 0.02  # seconds between state updates

        # Angle at which to fail the episode
        theta_threshold_radians = 12 * 2 * math.pi / 360
        x_threshold = 2.4

        x, x_dot, theta, theta_dot = state

        already_done =  x < -x_threshold \
            or x > x_threshold \
            or theta < -theta_threshold_radians \
            or theta > theta_threshold_radians
        already_done = bool(already_done)

        if already_done:

            next_state = state
            reward = 0
            done = True

        else:

            force = force_mag if action==1 else -force_mag
            costheta = math.cos(theta)
            sintheta = math.sin(theta)
            temp = (force + polemass_length * theta_dot * theta_dot * sintheta) / total_mass
            thetaacc = (gravity * sintheta - costheta* temp) / (length * (4.0/3.0 - masspole * costheta * costheta / total_mass))
            xacc  = temp - polemass_length * thetaacc * costheta / total_mass
            x  = x + tau * x_dot
            x_dot = x_dot + tau * xacc
            theta = theta + tau * theta_dot
            theta_dot = theta_dot + tau * thetaacc
            next_state = np.array([x,x_dot,theta,theta_dot])

            reward = 1

            done =  x < -x_threshold \
                or x > x_threshold \
                or theta < -theta_threshold_radians \
                or theta > theta_threshold_radians
            done = bool(done)

        return next_state, reward, done, {}
    
    def noisycartpole_reset(self):
        return self.cartpole_reset()

    def noisycartpole_step(self, state, action):

        next_state_mean, reward, done, info = self.cartpole_step(state, action)   # CartPole Step

        noise = np.zeros(self.stateDimension)
        if not done:
            noise = np.random.randn(self.stateDimension) * self.transitionSigmas        # Adding Noise
        next_state = next_state_mean + noise

        # logp = multivariate_normal.logpdf(next_state, mean=next_state_mean, cov=np.diagflat(self.transitionSigmas))

        return next_state, reward, done, None
    
    def reset(self):
        return self.noisycartpole_reset()
    
    def step(self, state, action):
        return self.noisycartpole_step(state, action)
    

In [3]:
env = NoisyCartPoleEnvironment()

# Utility Functions

In [4]:
def trajectory2tuples(states, actions):

    # Dimensions
    [sample_count, horizon, state_dimension] = states.shape
    [_, _, action_dimension] = actions.shape

    # Reshape Inputs and Targets
    inputs = np.reshape(states, (sample_count*horizon, state_dimension))
    targets = np.reshape(actions, (sample_count*horizon, action_dimension))

    return inputs, targets

In [5]:
def rollout_trajectories(env, policy, horizon, sample_count=1, init=None):

    # States and Actions
    states = np.zeros((sample_count, horizon, env.stateDimension))
    actions = np.zeros((sample_count, horizon, env.actionDimension))
    rewards = np.zeros((sample_count, horizon))
    
    # Sample Trajectories
    for t in range(horizon):

        # Initialization
        if t == 0:
            if init is None:
                states[:,t,:] = [ env.reset() for i in range(sample_count) ]
            else:
                states[:,t,:] = init
                
        # Transition and Reward
        else:
            for s in range(sample_count):
                states[s, t, :], rewards[s,t-1], _1, _2 = env.step(states[s, t-1, :], actions[s, t-1, :])
        
        actions[:,t,:] = policy.query(states[:, t, :]).reshape(sample_count, env.actionDimension)
        
    for s in range(sample_count):
        _, rewards[s, horizon-1], _1, _2 = env.step(states[s, horizon-1, :], actions[s, horizon-1, :])

    return states, actions, rewards

# Monte Carlo Sampling Method

In [9]:
def proposal_prior(_1, _2, _3, policy):
    return rollout_trajectories(env, policy, horizon)

In [35]:
def proposal_prior_cached(_1, _2, _3, policy):
    global states_cached, actions_cached, rewards_cached, cache_index
    
    if np.sum(states_cached) == 0:
        states_cached, actions_cached, rewards_cached = \
            rollout_trajectories(env, policy, horizon, sample_count=sample_count + burn_in)
    
    cache_index +=1
    
    return states_cached[cache_index,:,:], actions_cached[cache_index,:,:], rewards_cached[cache_index,:]

In [11]:
def proposal_keepthenprior(states, actions, rewards, policy):
    
    t = np.random.randint(states.shape[0]+1) -1
    
    if t < 0:
        states, actions, rewards = \
            rollout_trajectories(env, policy, horizon )
    else:
        states[t:,:], actions[t:,:], rewards[t:] = \
            rollout_trajectories(env, policy, horizon-t, init=states[t])
    
    return states, actions, rewards 

In [18]:
def mcmc(env, policy, proposal=proposal_prior):
    
    # States and Actions
    states = np.zeros((sample_count + burn_in, horizon, env.stateDimension))
    actions = np.zeros((sample_count + burn_in, horizon, env.actionDimension))
    rewards = np.zeros((sample_count + burn_in, horizon))

    for i in range(sample_count + burn_in):
        
        if i == 0:
            states[i,:,:], actions[i,:,:], rewards[i,:] = \
                rollout_trajectories(env, policy, horizon)
        else:
            states_candidate, actions_candidate, rewards_candidate = \
                proposal(states[i-1,:,:], actions[i-1,:,:], rewards[i-1,:], policy)
                
            reward_previous = np.sum(rewards[i-1,:])    
            reward_candidate = np.sum(rewards_candidate)

            alpha = reward_candidate / reward_previous

            if np.random.rand() < alpha:
                states[i,:,:] = states_candidate
                actions[i,:,:] = actions_candidate
                rewards[i,:] = rewards_candidate
            else:
                states[i,:,:] = states[i-1,:,:]
                actions[i,:,:] = actions[i-1,:,:]
                rewards[i,:] = rewards[i-1,:]
        
         # Resampling
        index = np.random.choice(range(sample_count), size=policy_sample_count, replace=False)

        # Policy Trajectories
        states_new = np.zeros((policy_sample_count, horizon, env.stateDimension))
        actions_new = np.zeros((policy_sample_count, horizon, env.actionDimension))
        rewards_new = np.zeros((policy_sample_count, horizon))
        for s in range(policy_sample_count):
            states_new[s] = states[index[s] + burn_in, :, :]
            actions_new[s] = actions[index[s] + burn_in, :, :]
            rewards_new[s] = rewards[index[s] + burn_in, : ]
        
    return states_new, actions_new, rewards_new

# Policy

In [19]:
class SciKitPolicy():

    def __init__(self):
        raise NotImplementedError()

    def query(self, states):
        if len(states.shape) == 1:
            states = states.reshape(1, -1)
        return self.method.predict(states)

    def train(self, inputs, targets):
        self.method.fit(inputs, targets)

    def m_step(self, states, actions):

        # States/Actions -> Inputs/Targets
        inputs, targets = trajectory2tuples(states, actions)

        # Train kNN
        self.train(inputs, targets.ravel())

In [20]:
class KnnPolicyDiscrete(SciKitPolicy):
    def __init__(self, k, weights='distance'):
        self.method = KNeighborsClassifier(n_neighbors=k, weights=weights, n_jobs=1)

In [21]:
class UniformPolicyDiscrete():

    def __init__(self, choices):
        self.choices = choices

    def query(self, states):
        return np.random.choice(self.choices, size=states.shape[0])

# Parameters

In [22]:
# Environment  
horizon = 100

# Inference
iteration_count = 2
sample_count = 1000
burn_in = 100

# Policy
kNearest = 5
policy_sample_count = 100
selectedPolicy = KnnPolicyDiscrete(kNearest)

# RLEMMC

In [23]:
%%prun
iteration_rewards = []

for i in range(iteration_count):

    if i == 0:
        iteration_policy = UniformPolicyDiscrete(env.actionSpace)
    else:
        iteration_policy = selectedPolicy

    # E-Step
    [states, actions, rewards] = mcmc(env, iteration_policy)

    # M-Step
    selectedPolicy.m_step(states, actions)
        
    # Average Reward
    iteration_rewards.append(np.mean(rewards) * horizon)
    print( f'Iteration {i+1} - Average Reward : {iteration_rewards[i]}' )

Iteration 1 - Average Reward : 26.56
Iteration 2 - Average Reward : 29.67
 

# Profiling

In [ ]:
%load_ext line_profiler

In [ ]:
# %lprun -f mcmc [states, actions, rewards] = mcmc(env, iteration_policy)

In [ ]:
# %lprun -f  rollout_trajectories \
#                states_candidate, actions_candidate, rewards_candidate = \
#                proposal_prior(states[0,:,:], actions[0,:,:], rewards[0,:], iteration_policy)

In [30]:
%time [states, actions, rewards] = mcmc(env, iteration_policy, proposal=proposal_prior)

Wall time: 41.2 s


In [31]:
cache_index = -1
states_cached = np.zeros((sample_count + burn_in, horizon, env.stateDimension))
actions_cached = np.zeros((sample_count + burn_in, horizon, env.actionDimension))
rewards_cached = np.zeros((sample_count + burn_in, horizon))

In [36]:
%time [states, actions, rewards] = mcmc(env, iteration_policy, proposal=proposal_prior_cached)

Wall time: 1.16 s


In [37]:
%time [states, actions, rewards] = mcmc(env, iteration_policy, proposal=proposal_keepthenprior)

Wall time: 18.6 s
